The objective of this notebook is to create files with the vectorized representation of the words for three different types: BOW, TF-IDF, word2vec to save time and resources (my pc)

In [15]:
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()
import pickle

In [16]:
df = pd.read_csv("../../data/silver/df_preprocessed.csv")
df

,Category,Message,word_count,char_count
0,0,go jurong point crazy available bugis n great ...,20,111
1,0,ok lar joking wif u oni,6,29
2,1,free entry number wkly comp win fa cup final t...,28,155
3,0,u dun say early hor u c already say,11,49
4,0,nah dont think go usf life around though,13,61
...,...,...,...,...
5149,1,numbernd time tried number contact u u poundnu...,30,160
5150,0,b going esplanade fr home,8,36
5151,0,pity mood soany suggestion,10,57
5152,0,guy bitching acted like id interested buying s...,26,125


## BOW

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

In [18]:
cv = CountVectorizer(ngram_range= (1,3))
		
bow_matrix = cv.fit_transform(df['Message'])

vocab = cv.get_feature_names_out()
print(vocab)

['aa' 'aa exhaust' 'aa exhaust hanging' ... 'zyada' 'zyada kisi'
 'zyada kisi ko']


In [19]:
with open('../../data/gold/bow/count_vectorizer.pkl', 'wb') as f:
    pickle.dump(cv, f)

with open('../../data/gold/bow/bow_matrix.pkl', 'wb') as f:
    pickle.dump(bow_matrix, f)

with open('../../data/gold/bow/vocab.pkl', 'wb') as f:
    pickle.dump(vocab, f)

## TF-IDF

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=1000)

In [21]:
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Message'])


In [22]:
vocab = tfidf_vectorizer.get_feature_names_out()

In [23]:
with open('../../data/gold/tfidf/tfidf_matrix.pkl', 'wb') as file:
    pickle.dump(tfidf_matrix, file)

with open('../../data/gold/tfidf/tfidf_vectorizer.pkl', 'wb') as file:
    pickle.dump(tfidf_vectorizer, file)

## Word2Vec

In [ ]:
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/maldu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
model = Word2Vec(
    sentences=df['Tokenized_Message'],  # Tu texto tokenizado
    vector_size=100,                    # Dimensión de los vectores
    window=5,                           # Tamaño de la ventana de contexto
    min_count=1,                        # Incluir todas las palabras
    workers=4                           # Número de hilos
)

In [ ]:
model.save("word2vec.model")

def get_average_word2vec(tokens_list, model, vector_size=100):
    valid_words = [word for word in tokens_list if word in model.wv]
    if valid_words:
        word_vectors = [model.wv[word] for word in valid_words]
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(vector_size)

df['Word2Vec_Avg'] = df['Tokenized_Message'].apply(lambda x: get_average_word2vec(x, model))


In [ ]:
with open('word2vec_model.pkl', 'wb') as f:
    pickle.dump(model, f)

df.to_pickle('word2vec_vectors.pkl')